In [37]:
import numpy as np 
import matplotlib.pyplot as plt 

prognosis_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/prognosis/pre_treatment_hamd_reduction_50/'

data = np.load(prognosis_path + 'hb_data.npy')
label = np.load(prognosis_path + 'label.npy')



# input: prognosis/ hb_data -> shape (65, 52, 2502)

# process 

    1 - std > 0.1 -> set to 0 
    1.5 - similarity < 0 -> set to 0
    2 - all channel will be avg with surrouding signal 
    3. hbo and hbr will be performed above mentioned steps seperatelly, and then hbt will be calculated 


# output: shape (65, 52, 1250, 3) last dimension is (hbo, hbr, hbt)
# note:
    - neighouborhood averaging is a good strategy to get more useful signal. as many channels were canceled and set to 0 because of std>0.1 and similarity<0
    1. hbt result is not so good





In [51]:
import numpy as np
adj_matrix = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/neighbour_matrix.npy')[0]
print(adj_matrix.shape)

sample = np.arange(52)

# plt.imshow(adj_matrix[0])

# argmax = np.argmax(adj_matrix[0], axis=1)
selected_sample = sample[adj_matrix[0] == 1]
print(selected_sample)



# tmp = average_neighbour_channel(hbo, adj_matrix)

(52, 52)
[ 0  1 10 11]


In [ ]:
print(data.shape)
hbo = data[:, :, :1250].copy()
hbr = data[:, :, 1252:].copy()
print(hbr.shape)


def compare_signals(signal1, signal2):
    correlation = np.corrcoef(signal1, signal2)[0, 1]
    return correlation
def set_to_zero_if_std_greated_than_threshold(data, threshold):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if np.std(data[i,j,:]) > threshold:
                data[i,j,:] = 0
    return data
def set_to_zero_if_similarity_lower_than_threshold(data, compared_data, threshold):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            similarity = compare_signals(data[i,j,:], compared_data)
            if similarity < threshold:
                data[i,j,:] = 0
    return data
def average_neighbour_channel(data, adj_matrix):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i, j] = np.mean(data[i, adj_matrix[j] == 1], axis=0)
    return data 



data.shape

hbr_std_threshold = hbo_std_threshold = 0.1
hbr_similiarity_threshold = hbo_similiarity_threshold = 0

hbo = set_to_zero_if_std_greated_than_threshold(hbo, hbo_std_threshold)
mean_hbo = np.mean(hbo, axis=(0,1))
hbo = set_to_zero_if_similarity_lower_than_threshold(hbo, mean_hbo, hbo_similiarity_threshold)


hbr = set_to_zero_if_std_greated_than_threshold(hbr, hbr_std_threshold)
mean_hbr = np.mean(hbr, axis=(0,1))
hbr = set_to_zero_if_similarity_lower_than_threshold(hbr, mean_hbr, hbr_similiarity_threshold)
hbr = average_neighbour_channel(hbr, adj_matrix)

plot_data = hbr 
plot_mean_data = mean_hbr

for subject in [0,1,2,3]:
    fig, axs = plt.subplots(7,8, figsize=(20,20))
    for i in range(52):
        x = i//8 
        y = i%8
        axs[x,y].plot(plot_data[subject, i])
        std = np.std(plot_data[subject, i])
        similarity = compare_signals(plot_data[subject, i], plot_mean_data)
        axs[x,y].set_title('std:{:0.2f}si:{:0.3f}'.format(std,similarity))
    plt.tight_layout()



In [ ]:
hbt = hbo + hbr
mean_hbt = np.mean(hbt, axis=(0,1))

plot_data = hbt 
plot_mean_data = mean_hbt

for subject in [0,1,2,3]:
    fig, axs = plt.subplots(7,8, figsize=(20,20))
    for i in range(52):
        x = i//8 
        y = i%8
        axs[x,y].plot(plot_data[subject, i])
        std = np.std(plot_data[subject, i])
        similarity = compare_signals(plot_data[subject, i], plot_mean_data)
        axs[x,y].set_title('std:{:0.2f}si:{:0.3f}'.format(std,similarity))
    plt.tight_layout()



In [55]:
result_data = np.concatenate([hbo[..., np.newaxis], hbr[..., np.newaxis], hbt[..., np.newaxis]], axis=-1)
print(result_data.shape)
np.save(prognosis_path+'hb_data_v1', result_data)

(65, 52, 1250, 3)
